In [23]:
import os
import subprocess as sp
import yaml
import shutil
from tqdm import tqdm
import numpy as np
import datetime
import re

# Convert notebooks to html

In [24]:
os.chdir('/Users/ginoprasad/ginoprasad.github.io')

In [25]:
metadata_path = 'metadata.yaml'
with open(metadata_path) as infile:
    metadata = yaml.safe_load(infile)

In [26]:
for project_notebook_path in metadata['Projects'][:]:
    if not os.path.exists(project_notebook_path):
        print(f"REMOVING {project_notebook_path}")
        metadata['Projects'].remove(project_notebook_path)
        with open(metadata_path, 'w') as outfile:
            yaml.dump(metadata, outfile, default_flow_style=False)

In [27]:
temp_path = f'{os.getcwd()}/projects/temp.html'

In [28]:
max_base_filename_length = 50

In [29]:
def get_notebook_metadata(project_notebook_path):
    notebook_metadata_str = sp.run(f"head -n 200 '{project_notebook_path}'", shell=True, capture_output=True).stdout.decode().split('\n')
    notebook_metadata_str = ''.join(notebook_metadata_str[notebook_metadata_str.index('   "source": [')+1:notebook_metadata_str.index('   ]')])
    
    notebook_metadata = {}
    notebook_metadata['title'] = re.search('(?<=# ).*?(?<=\\\\n)', notebook_metadata_str).group(0)[:-2]
    notebook_metadata['authors'] = re.search('(?<=#### ).*?(?<=\\\\n)', notebook_metadata_str).group(0)[:-2]
    notebook_metadata['date'] = re.search('[0-9][0-9]/[0-9][0-9]/[0-9][0-9][0-9][0-9]', notebook_metadata_str).group(0)
    
    return notebook_metadata

In [30]:
datetime.datetime.now().strftime('%D')

'01/27/24'

In [31]:
project_names, project_paths, project_dates = [], [], []
for project_notebook_path in tqdm(metadata['Projects']):
    notebook_metadata = get_notebook_metadata(project_notebook_path)    
    project_base_path = os.path.basename(project_notebook_path)[:-len('.ipynb')]

    while len(project_base_path) > max_base_filename_length:
        project_base_path = ' '.join(project_base_path.split(' ')[:-1])
    if not project_base_path:
        print(f"\n\n\n\n\tWarning: Project '{notebook_metadata['title']}' Name exceeds recommended length\n\n\n\n")
        project_base_path = notebook_metadata['title']
    notebook_metadata['project_path'] = f'{os.getcwd()}/projects/{project_base_path}.html'
    
    assert notebook_metadata['project_path'] not in project_paths
    project_names.append(notebook_metadata['title'])
    project_paths.append(notebook_metadata['project_path'])
    project_dates.append(notebook_metadata['date'])
    
    if os.path.exists(notebook_metadata['project_path']) and os.path.getmtime(notebook_metadata['project_path']) > os.path.getmtime(project_notebook_path):
        continue
    
    print(project_base_path)
    print(f"Converting {project_notebook_path}")
    sp.run(f"jupyter nbconvert --to html '{project_notebook_path}' --output '{temp_path}'", shell=True)
    print(f"Project Name: {notebook_metadata['title']}")

    with open(temp_path) as infile:
        lines = infile.readlines()


    title = ' '.join(map(lambda x: x[0].upper() + x[1:] if x else x, project_base_path.split('_')))
    lines[5] = lines[5][:len('<title>')] + title + lines[5][lines[5].index('</title>'):]
 
    with open(temp_path, 'w') as outfile:
        lines.insert(5, '<link rel="icon" href="../docs/assets/logo.png"><iframe src="../header.html" style="height: 12rem; width: 100%" frameborder="0" scrolling="no"></iframe>\n')
        outfile.write(''.join(lines))
    os.rename(temp_path, notebook_metadata['project_path'])
    
    print('\n')

  0%|                                    | 0/22 [00:00<?, ?it/s]

earth_movers_distance
Converting /Users/ginoprasad/ai_stuff/earth_movers_distance.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/ai_stuff/earth_movers_distance.ipynb to html
[NbConvertApp] Writing 611487 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html
  5%|█▎                          | 1/22 [00:01<00:39,  1.88s/it]

Project Name: Earth Mover's Distance for NLP using Network Simplex


transformer_seq_to_seq
Converting /Users/ginoprasad/ai_stuff/Translation/transformer_seq_to_seq.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/ai_stuff/Translation/transformer_seq_to_seq.ipynb to html
[NbConvertApp] Writing 658412 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html
  9%|██▌                         | 2/22 [00:03<00:36,  1.84s/it]

...

Project Name: Wordle Bot


interpolation
Converting /Users/ginoprasad/eng_math/interpolation.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/eng_math/interpolation.ipynb to html
[NbConvertApp] Writing 585551 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html
 95%|█████████████████████████▊ | 21/22 [00:31<00:01,  1.19s/it]

Project Name: Lagrange Interpolation and Gregory-Newton Interpolation


edge_detector
Converting /Users/ginoprasad/ai_stuff/edge_detector.ipynb


[NbConvertApp] Converting notebook /Users/ginoprasad/ai_stuff/edge_detector.ipynb to html
[NbConvertApp] Writing 590889 bytes to /Users/ginoprasad/ginoprasad.github.io/projects/temp.html
100%|███████████████████████████| 22/22 [00:32<00:00,  1.47s/it]

Project Name: Image Edge Detector




In [37]:
datetimes = [datetime.datetime.strptime(project_date, '%m/%d/%Y') for project_date in project_dates]
project_names = sorted(project_names, key=datetimes)
project_dates = sorted(project_dates, key=datetimes)
datetimes

TypeError: 'list' object is not callable

In [9]:
project_names

["Earth Mover's Distance for NLP using Network Simplex",
 'English to French Translation using Transformers',
 'Variational Auto Encoders for Simulating Human Faces',
 'MUS 15 Project 2: Audio Transcription',
 'Cropping Image Logo for the Website',
 'Using Pretrained Stable Diffusion for the Website Logo',
 'Google Foobar Expanding Nebula Problem',
 'Inner Product in Hilbert Space',
 'MUS 15 Project 1: Creating Autotune From Scratch Using Phase Vocoders',
 'Phase Vocoder Implementation',
 'Yin Pitch Prediction Implementation',
 'Autotune Implementation Using Phase Vocoder',
 'Fast Fourier Transform Implementation',
 'U-Net Convolutional Neural Network Implementation',
 'Multilayer Neural Network Implementation From Scratch: Handwritten Text Parser',
 'Image Segmenter Using Connected Components',
 'Perceptron Implementation From Scratch: Handwritten Text Parser',
 'Gaussian Filter Implementation',
 'Pitch Reconstruction with Audio Processing',
 'Wordle Bot',
 'Lagrange Interpolation and

In [9]:
index_html_path = 'index.html'
index_html_lines = open(index_html_path).readlines()

In [15]:
project_template = "\t\t\t<li><div class=link><a href=\"{}\">{}</a></div><div class='date'><img src='docs/assets/calendar_icon.png'><span class=date>{}</span></div></li>\n"

project_list_index_start = ["Cool Projects" in x for x in index_html_lines].index(True) + 2
project_list_index_end = index_html_lines[project_list_index_start:].index('\t\t</ul>\n') + project_list_index_start

new_project_list =  [project_template.format(os.path.basename(html_path), name, date) for name, html_path, date in zip(project_names, project_paths, project_dates)]
index_html_lines = index_html_lines[:project_list_index_start] + new_project_list + index_html_lines[project_list_index_end:]
index_html_lines[project_list_index_start-2] = f"\t\t<h2> Cool Projects ({len(metadata['Projects'])}) </h2>\n"

# Copying CV and Updating Links

In [16]:
assert shutil.copy(metadata['CV'], f"projects/{os.path.basename(metadata['CV'])}")

In [17]:
tag_dict = {tag: metadata[tag] for tag in ['CV', 'LinkedIn', 'GitHub']}
tag_dict['CV'] = f"projects/{os.path.basename(tag_dict['CV'])}"

In [18]:
for i, line in enumerate(index_html_lines):
    for tag in tag_dict:
        prefix = f"<a id='{tag}' href='"
        if line.startswith(prefix):
            print(line.strip())
            new_line = prefix + tag_dict[tag] + line[len(prefix) + line[len(prefix):].index("'"):]
            print(new_line)
            index_html_lines[i] = new_line
    
    if line.startswith(prefix):
        del tag_dict[tag]

# Writing Updated Index File

In [19]:
with open(index_html_path, 'w') as outfile:
    outfile.write(''.join(index_html_lines))

In [20]:
sp.run(f"cd '{os.getcwd()}'; git add .; git commit -m 'Automated Website Update'; git push origin main", shell=True)

[main 55e4115] Automated Website Update
 20 files changed, 61174 insertions(+), 184 deletions(-)
 create mode 100644 docs/assets/calendar_icon.png
 create mode 100644 projects/Gaussian_Blur.html
 create mode 100644 projects/Handwriten_Text_Parser.html
 create mode 100644 projects/Image_Segmenter.html
 create mode 100644 projects/Multilayer_Neural_Network.html


To https://github.com/GinoP123/ginoprasad.github.io.git
   361b54c..55e4115  main -> main


CompletedProcess(args="cd '/Users/ginoprasad/ginoprasad.github.io'; git add .; git commit -m 'Automated Website Update'; git push origin main", returncode=0)

# Updating Python Script

In [22]:
if hasattr(__builtins__,'__IPYTHON__'):
    sp.run(f"jupyter nbconvert --to script 'update_website.ipynb' --output 'update_website'", shell=True)

[NbConvertApp] Converting notebook update_website.ipynb to script
[NbConvertApp] Writing 5603 bytes to update_website.py
